
<h1 style="text-align:center">   
      <font color = red >
                Netflix Movies and TV Shows - Data Analysis and Visualization 
        </font>    
</h1>   
<hr style="width:100%;height:5px;border-width:0;color:gray;background-color:gray">
<center><img style = "height:450px;" src="https://media.npr.org/assets/img/2019/07/04/ap_19165551810863-852cd5b90680e782e02303308afd219172d04aa1-s800-c85.jpg"></center>


## INTRODUCTION
Netflix is a streaming service that offers a wide variety of award-winning TV shows, movies, anime, documentaries, and more on thousands of internet-connected devices.

### CONTENT 
  
1. [Load and Check Data](#1)
1. [Variable Description](#2) 
1. [Data Cleaning](#3)
    * [Find Missing Values](#4)
    * [Fill Missing Values](#5)
1. [Visualization](#6) 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import warnings
warnings.filterwarnings('ignore')
                        
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id = "1" ></a>
<font color = 'red'>
# Load and Check Data

In [ ]:
# Read data
netflix_df = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")

In [ ]:
netflix_df.head()

In [ ]:
netflix_shows=netflix_df[netflix_df['type']=='TV Show']


In [ ]:
netflix_movies=netflix_df[netflix_df['type']=='Movie']

<a id = "2"></a>
<font color = 'red'>
# Variable Description
<font color = 'grey'>
    
1. Show_id : Unique ID for every Movie / Tv Show
1. Type : Identifier - A Movie or TV Show
1. Title : Title of the Movie / Tv Show
1. Director: Director of the Movie
1. Cast : Actors involved in the movie / show
1. Country : Country where the movie / show was produced 
1. Date_added : Date it was added on Netflix
1. Release_year :Actual Release year of the move / show
1. Rating: TV Rating of the movie / show
1. Duration: Total Duration - in minutes or number of seasons
1. Listed_in : Genre
1. Description: The summary description

In [ ]:
netflix_df.info()

<a id = "3"></a>
<font color = 'red'>
# Data Cleaning

<a id = "4"></a>
## Find Missing Values

In [ ]:
#Checking for null values
netflix_df.isnull().sum() 

In [ ]:
#Checking for unique values
netflix_df.nunique()

In order to visualize missing data, we can use :
* sns.heatmap 
* msno.bar 

In [ ]:
sns.heatmap(netflix_df.isnull(),cmap = 'Reds')
plt.title('Null Values Heatmap')
plt.show()

There are a few columns that contain null values ('director', 'cast', 'country', 'date_added', 'rating').

In [ ]:
import missingno as msno
msno.bar(netflix_df,color = 'indianred')
plt.show()

Numbers at left side of plot is percentage.<br>
Numbers at top of the plot is number of non missign values.<br>
Numbers at right side of plot is number of sample in data.

<a id = "5"></a>
## Fill Missing Values

<ul>
    <li>Drop id column</li>
    <li>For the missing rows in date_added column, replace them by January 1,{release_year}</li>
    <li>We have 10 missing values in rating column, replace them with TV-MA which is the most common rating</li>
    <li>For the missing values in country, replace them with United States.Instead of removing the entire column, we chose to fill in the missing values in this way to avoid losing data. 
    <li>Drop the director and cast columns completely. We are only focused on visualizing this data hence dropping two columns wont be any trouble for us.But this should not be a regular practise as if we are making a recommender system, we cannot drop the director and cast of a movie as these are a key feature used to recommend movies to users.    
</ul>

In [ ]:
# drop id column
netflix_df.drop('show_id',axis=1, inplace = True)

In [ ]:
# Replace nan values in data_added with January 1,{release_year}
netflix_df['date_added']=netflix_df['date_added'].fillna('January 1, {}'.format(str(netflix_df['release_year'].mode()[0])))

In [ ]:
netflix_df['date_added'].isnull().sum() 

In [ ]:
netflix_df["release_month"] = netflix_df['date_added'].apply(lambda x: x.lstrip().split(" ")[0])
netflix_df.head()

In [ ]:
netflix_df['rating'].value_counts()

In [ ]:
#  Replace nan values in rating with TV-MA
netflix_df['rating'].replace(np.nan, 'TV-MA',inplace  = True)

In [ ]:
netflix_df['country'].value_counts()

In [ ]:
# Replace nan values in country with United States
netflix_df['country'].replace(np.nan, 'United States',inplace  = True)

In [ ]:
# Drop the director and cast columns completely.
netflix_df.drop(['director','cast'],axis=1, inplace = True)

In [ ]:
netflix_df.isnull().sum()

In [ ]:
netflix_df.head()

<a id = "6"></a>
<font color = 'red'>
# Visualization

### Analysis of Movies vs TV Shows.

In [ ]:
print(netflix_df.type.value_counts())
sns.countplot(netflix_df.type,palette="pastel")
plt.show()

### Analysis of Ratings

In [ ]:
rating_counts = netflix_df.rating.value_counts()
print(rating_counts)
plt.figure(figsize = (12,8))
sns.countplot(netflix_df.rating, order = rating_counts.index[0:15],palette="pastel")
plt.title("Ratings for Movies And Shows")
plt.xlabel("Rating")
plt.ylabel("Total Count")
plt.show()

> 1 -Programming rated TV-MA in the United States by the TV Parental Guidelines signifies content for mature audiences.
> 
> 2 -Programming rated TV-14 in the United States TV Parental Guidelines signifies content with parents strongly cautioned. Content may be inappropriate for children younger than 14 years of age.
> 
> 3 -An R-rated film is a film that has been assessed as having material which may be unsuitable for children under the age of 17.

In [ ]:
explode = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
sizes = rating_counts.values

# visual
plt.figure(figsize = (10,10))
plt.pie(sizes, explode=explode, labels=rating_counts.index, autopct='%1.1f%%')
plt.title('Ratings for Movies And Shows',fontsize = 15)
plt.show()

In [ ]:
#Type - rating 
plt.figure(figsize = (12,8))
sns.countplot(x='rating',data = netflix_df,hue='type',palette="pastel")
plt.xlabel("Rating")
plt.ylabel("Total Count")
plt.show()

### Year wise analysis

In [ ]:
release_year_counts = netflix_df.release_year.value_counts()
print(release_year_counts)
plt.figure(figsize = (35,8))
sns.countplot(netflix_df.release_year, order = release_year_counts.index[0:94],palette="pastel")
plt.show()

> As we can see most of the movies and Tv shows on Netflix are released in 2018.Let's see which month directors prefer most to release their Movies & Tv Shows.

In [ ]:
plt.figure(figsize=(20,6))
sns.countplot(x="release_month",data= netflix_df,order = netflix_df['release_month'].value_counts().index[0:12],palette="pastel")
plt.xticks(rotation=45)
plt.show()

### Countries with the most content available

In [ ]:
print(netflix_df["country"].value_counts().head())
plt.figure(figsize=(20,6))
sns.countplot(x="country",data= netflix_df,hue= "type",order = netflix_df['country'].value_counts().index[0:15],palette="pastel")
plt.xticks(rotation=45)
plt.show()

> Unsurprisingly, the United States stands out because Netflix is an American company. India surprisingly ranks second in the film, followed by the UK.

### Top 10 Genres of Movies

In [ ]:
print(netflix_movies["listed_in"].value_counts()[:10])
plt.figure(figsize=(12,6))
sns.countplot(y='listed_in',data = netflix_movies,order =netflix_movies["listed_in"].value_counts().index[0:10],palette="pastel")
plt.title("Top 10 Genres of Movies",size=18)
plt.show()


> Let's see the same thing in tv shows

### Top 10 Genres of Tv Shows

In [ ]:
print(netflix_shows["listed_in"].value_counts()[:10])
plt.figure(figsize=(12,6))
sns.countplot(y='listed_in',data = netflix_shows,order =netflix_shows["listed_in"].value_counts().index[0:10],palette="pastel")
plt.title("Top 10 Genres of Tv Shows",size=18)
plt.show()


### Analysis of duration of movies and Tv Shows

In [ ]:
netflix_movies.duration = netflix_movies.duration.str.replace(' min','').astype(int)
netflix_shows.rename(columns={'duration':'seasons'}, inplace=True)
netflix_shows.replace({'seasons':{'1 Season':'1 Seasons'}}, inplace=True)
netflix_shows.seasons = netflix_shows.seasons.str.replace(' Seasons','').astype(int)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(19, 5))
g1 = sns.distplot(netflix_movies.duration,ax=ax[0]);
g1.set_xticks(np.arange(0,360,30))
g1.set_title("Duration Distribution for Netflix Movies")
g1.set_ylabel("% of All Netflix Movies")
g1.set_xlabel("Duration (minutes)")
g2 = sns.countplot(netflix_shows.seasons,ax=ax[1],palette="pastel");
g2.set_title("Netflix TV Shows Seasons")
g2.set_ylabel("Count")
g2.set_xlabel("Season(s)")
fig.show()

> As you can see, movies are usually between 75-120 minutes and TV shows are usually 1 season.

### Oldest movies available on netflix

In [ ]:
oldest = netflix_df.sort_values("release_year", ascending = True) 
oldest[['title', "release_year"]][:10]

![](https://data.pixiz.com/output/user/store/9/f/7/0/9fd2aea83ccc1ab6035e601615ee07f9.jpg)

### Standup shows on Netflix


In [ ]:
standup=netflix_df[netflix_df["listed_in"] == "Stand-Up Comedy"]
standup[["title","country","release_year"]].head(10)

![](https://data.pixiz.com/output/user/store/6/3/e/5/f633ac39e4c7cb37aa01147f12295e36.jpg)

### Kids TV shows on Netflix

In [ ]:
kids=netflix_df[netflix_df["listed_in"] == "Kids' TV"]
kids[["title","country","release_year"]].head(10)

![](https://data.pixiz.com/output/user/store/b/5/e/5/f854582f81028d73ea8e75cbd16e5e5b.jpg)

If you like the kernel,please upvote ! 